In [1]:
%matplotlib inline

import os 
import sys 
import time
import hashlib

from datetime  import datetime 

import numpy as np  # learn 
import pandas as pd # learn
from pandas import *
from numpy import *

from scipy import stats # look at scipy
from scipy import linalg
from scipy import *

 
import matplotlib as mplt # learn matplolib 
from matplotlib import cm
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d import proj3d

import seaborn as sns 
sns.set_style("whitegrid")
sns.set(rc={"figure.figsize": (14, 6)})

import IPython
from IPython.display import display


import matplotlib
from matplotlib.ticker import FuncFormatter
import matplotlib.pyplot as plt
 
import sklearn as sk
import itertools



from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext
from pyspark.sql.types import *

from pyspark.mllib.regression import LabeledPoint
import random
import time


In [3]:
#BT data

# dta_RDD = sc.textFile("file:///home/tguo/data/tian-test/udpjitter-H1april2014_regTree.csv")

dta_RDD = sc.textFile("hdfs://computer61.ant-net/user/tguo/udpjitter-H1april2014_regTree_mllib.csv")


dta_splited = dta_RDD.map(lambda line: line.split(",")).map(lambda r:
                                                                (float(r[0]),int( float(r[1])), 
                                                                 int(float(r[2])),int(float(r[3])),
                                                                 int(float(r[4])),int(float(r[5])),
                                                                 int(float(r[6])),int(float(r[7])) ))
# dta_splited.first()

dta= dta_splited
b
dta.cache()
dta.first()

# print dta.count()

In [323]:
# dta_withNodeIdx = dta_splited.map( lambda r: (0, [r[0], r[1], r[2],r[3],r[4],r[5],r[6],r[7] ] ))

# print dta_withNodeIdx.first()

In [324]:
# for test: randomly assign node index to data points and then aggregate the information of each node
# dta_withRandomNodeIdx = dta_splited.map( lambda r: (random.randint(0, 7), 
#                                                     [r[0], r[1], r[2],r[3],r[4],r[5],r[6],r[7] ] ))

# print dta_withRandomNodeIdx.first()


In [13]:
#Flight data

dta_flight = sc.textFile("file:///home/tguo/data/tian-test/2008_flight_mllib.csv")

dta_flight = dta_flight.cache()
# dta.first()

# dta_csv.iloc[0]


dta_splited = dta_flight.map(lambda line: line.split(",")).map(lambda r:
                                                                 (float(r[0]),int( float(r[1])), 
                                                                  int(float(r[2])),int(float(r[3])),
                                                                  int(float(r[4])),int(float(r[5])),
                                                                  int(float(r[6])),int(float(r[7])) ))
# # dta_splited.first()

dta= dta_splited

# dta.cache()
print dta.first()
print dta.count()


# dta_filtered = dta.filter(lambda line: line[0] <2000 )
# dta_filtered.cache()
# dta_filtered.count()

(-14.0, 1, 3, 4, 0, 0, 0, 0)
7009728


In [26]:
maxdepth=5
numFeatures=7
# numFeatureValues = 30
numRandomNode=63

# assign data to new nodes
# def search_nodeToData(line, tree):
#     for i in range(0,maxdepth):
#         if line[0]>10:
#             tmp=line[0]
#         else:
#             tmp=line[0]
#     return random.randint(0, numRandomNode)        

# dta_test = dta.map( lambda line: (search_nodeToData(line, tree),(line
#                                                                 )) ) 

# dta_test=node_dta.zipWithIndex(dta)

In [690]:
# tmprdd= dta.map(lambda line: line[2])

# print tmprdd.distinct().collect()

In [326]:
# for debug
a={1:2}
# a.update({3:2})
a.update( { 3:{1:[2]}  }  )
a.update( { 1:{3:[2]}  }  )


a[3][1].append(4)
if 1 in a[3]:
    print 'fadf'
    
    
for i in a[1].keys():
    print i
    
def func(tmp_dict):
    tmop_dict.update({ 1:{ 1:[3]  }   })
    tmop_dict.update({ 1:{ 1:[3]  }   })


fadf
3


In [375]:
# basic functions in two-pass method

def ini_combiner(dvAndfeatures):
    # each feature-value has a triple [cnt, mean, variance]
    
    tmp_feature={}
    Y= dvAndfeatures[0]
    
    for i in range(0,numFeatures):
        feature_val=dvAndfeatures[i+1]
        tmp_feature.update( {i:{feature_val: [ 1, Y, 0] }   }   )
    return tmp_feature

def update_combiner(featureToValueToStat,dvAndfeatures):
    
    tmp_featureToValueToStat= featureToValueToStat
    
    Y= dvAndfeatures[0]
    for i in range(0, numFeatures):
        feature_val=dvAndfeatures[i+1]
    
        if feature_val in featureToValueToStat[i]:
            preCnt= featureToValueToStat[i][feature_val][0]
            preMean= featureToValueToStat[i][feature_val][1]
            preVar= featureToValueToStat[i][feature_val][2]
         
            tmp_featureToValueToStat[i][feature_val][2]=featureToValueToStat[i][feature_val][2] + preCnt *1.0/ (preCnt+1)*( preMean- Y)*( preMean- Y)
        
            tmp_featureToValueToStat[i][feature_val][1]= (1.0* preMean * preCnt + Y )/( 1.0*preCnt+1)
        
            tmp_featureToValueToStat[i][feature_val][0] =  preCnt  +1
        else:
            tmp_featureToValueToStat[i].update( {feature_val: [ 1, Y, 0] } )
            
    return tmp_featureToValueToStat

    
def merge_combiner(featureToValueToStat_1, featureToValueToStat_2):  
# merged in featureToValueToStat_1
    
    for i in range(0, numFeatures):
        
        for j in featureToValueToStat_1[i].keys():
            
            feature_val=j
            
            if feature_val in featureToValueToStat_2[i]:
                
                featureToValueToStat_1[i][j][2] =1.0* featureToValueToStat_1[i][j][2] + \
                featureToValueToStat_2[i][j][2] + \
                ( (1.0* featureToValueToStat_1[i][j][0]*featureToValueToStat_2[i][j][0] ) / (1.0*featureToValueToStat_1[i][j][0]+featureToValueToStat_2[i][j][0])* \
                 (1.0*featureToValueToStat_1[i][j][1] - featureToValueToStat_2[i][j][1])* (1.0*featureToValueToStat_1[i][j][1] - featureToValueToStat_2[i][j][1]))     
                
                
                featureToValueToStat_1[i][j][1] = (featureToValueToStat_1[i][j][1]*featureToValueToStat_1[i][j][0]+ \
                                           featureToValueToStat_2[i][j][1]*featureToValueToStat_2[i][j][0]*1.0)\
                                           /(1.0* featureToValueToStat_1[i][j][0]+featureToValueToStat_2[i][j][0]  )
                    
                featureToValueToStat_1[i][j][0] = featureToValueToStat_1[i][j][0]+featureToValueToStat_2[i][j][0]
    
    
    for i in range(0, numFeatures):
        
        for j in featureToValueToStat_2[i].keys():
            
            feature_val=j
            
            if feature_val not in featureToValueToStat_1[i]:
                
                tmp_cnt=featureToValueToStat_2[i][j][0]
                tmp_mean=featureToValueToStat_2[i][j][1]
                tmp_var=featureToValueToStat_2[i][j][2]
                
                featureToValueToStat_1[i].update({feature_val: [ tmp_cnt, tmp_mean, tmp_var] })
                    
    
    return  featureToValueToStat_1



    


In [ ]:
# two-pass method 

start = time.time()

# assign data to new nodes
def search_nodeToData(line, tree):
    for i in range(0,maxdepth):
        if line[0]>10:
            tmp=line[0]
        else:
            tmp=line[0]
    return random.randint(0, numRandomNode)        

dta_test = dta.map( lambda line: (search_nodeToData(line, tree),(line
                                                                )) ) 


nodeStatis = dta_test.combineByKey(lambda line:ini_combiner(line),
                             lambda nodeToFeatureToValue, line: (update_combiner(nodeToFeatureToValue,line)),
                             lambda nodeToFeatureToValue_x,nodeToFeatureToValue_y: (merge_combiner( nodeToFeatureToValue_x, nodeToFeatureToValue_y))
                            )

local_NodeFeatureValue_statis = nodeStatis.collect()


# run your code
end = time.time()
elapsed = end - start

print elapsed, 'sec'

In [377]:
# for debug

print len(local_NodeFeatureValue_statis)


64


In [15]:
# basic functions in one-pass method

# assign each data instance to the node
def search_nodeToData(features, tree):
        
    nodeNum=len(tree)
    
    if nodeNum == 0:
        return 0;
    
    current_nodeIdx=0
    while current_nodeIdx< nodeNum:
        split_feature= tree[current_nodeIdx][0]
        split_valueSet= tree[current_nodeIdx][1]
        
        if split_feature==-1:
            return -1
        
        if (( 1<<features[split_feature]) &  split_valueSet) != 0 :
            current_nodeIdx=current_nodeIdx*2+1
        else:
            current_nodeIdx=current_nodeIdx*2+2
    
#     return random.randint(0, 2)    
    return current_nodeIdx - nodeNum


def partition_combiner(list_dvAndfeatures):
    # each feature-value has a triple [cnt, mean, variance]
    
    nodes_dict={}
    tmpcnt=0
    
    for dvAndfeatures in list_dvAndfeatures:
        
        Y= dvAndfeatures[0]
        
        node= search_nodeToData(dvAndfeatures[1:numFeatures+1], \
                                currentNode_split_fromMaster.value)
        
        if node in nodes_dict.keys():
            
            # new added: sum of y in a node
            nodes_dict[node]['sumY'] = nodes_dict[node]['sumY']+Y 
            nodes_dict[node]['count'] = nodes_dict[node]['count']+1 
            
        
            for i in range(0,numFeatures):
                
                feature_val= dvAndfeatures[i+1]
                
                if feature_val not in nodes_dict[node][i]:
                    nodes_dict[node][i].update( { feature_val: [1,Y,0]   }  )
                else:
                    preCnt= nodes_dict[node][i][feature_val][0]
                    preMean= nodes_dict[node][i][feature_val][1]
                    preVar= nodes_dict[node][i][feature_val][2]
         
                    nodes_dict[node][i][feature_val][2]=preVar + \
                    preCnt *1.0/ (preCnt+1)*( preMean- Y)*( preMean- Y)
        
                    nodes_dict[node][i][feature_val][1]=(1.0*preMean*preCnt+Y)/(1.0*preCnt+1)
                    nodes_dict[node][i][feature_val][0] =  preCnt  +1
            
        else:
            nodes_dict.update( {node: {}})  
            
            # new added: sum of y in a node
            nodes_dict[node].update( { 'sumY': Y} )
            nodes_dict[node].update( { 'count': 1} )
            
            for i in range(0,numFeatures):
                feature_val= dvAndfeatures[i+1]
                nodes_dict[node].update( { i: {}} )    
                nodes_dict[node][i].update( { feature_val: [1,Y,0]   }  )
                
    return zip(nodes_dict.keys(), nodes_dict.values())
# , nodes_dict.values()


def merge_parttion_combiner(nodeToFeatureToValue_1,nodeToFeatureToValue_2 ):
    
    # new added: sum of Y in a node
    nodeToFeatureToValue_1['sumY']= nodeToFeatureToValue_1['sumY']+\
    nodeToFeatureToValue_2['sumY']
    nodeToFeatureToValue_1['count']= nodeToFeatureToValue_1['count']+\
    nodeToFeatureToValue_2['count'] 
    
    
    for i in range(0, numFeatures): #feature
        for j in nodeToFeatureToValue_1[i].keys(): #feature value
            feature_val=j
            if feature_val in nodeToFeatureToValue_2[i].keys():
                
                
                nodeToFeatureToValue_1[i][j][2] =1.0* nodeToFeatureToValue_1[i][j][2] + \
                nodeToFeatureToValue_2[i][j][2] + \
                ( (1.0* nodeToFeatureToValue_1[i][j][0]*nodeToFeatureToValue_2[i][j][0] ) / (1.0*nodeToFeatureToValue_1[i][j][0]+nodeToFeatureToValue_2[i][j][0])*\
                 (1.0*nodeToFeatureToValue_1[i][j][1] - nodeToFeatureToValue_2[i][j][1])* (1.0*nodeToFeatureToValue_1[i][j][1] - nodeToFeatureToValue_2[i][j][1]))   
                
                nodeToFeatureToValue_1[i][j][1] = (nodeToFeatureToValue_1[i][j][1]*nodeToFeatureToValue_1[i][j][0]*1.0+ \
                                                   nodeToFeatureToValue_2[i][j][1]*nodeToFeatureToValue_2[i][j][0]*1.0)/(1.0* nodeToFeatureToValue_1[i][j][0]+1.0*nodeToFeatureToValue_2[i][j][0])
                    
                nodeToFeatureToValue_1[i][j][0] = nodeToFeatureToValue_1[i][j][0]+nodeToFeatureToValue_2[i][j][0]
    
    for i in range(0, numFeatures):
            for j in nodeToFeatureToValue_2[i].keys():
            
                feature_val=j
            
                if feature_val not in nodeToFeatureToValue_1[i].keys():
                    tmp_cnt=nodeToFeatureToValue_2[i][j][0]
                    tmp_mean=nodeToFeatureToValue_2[i][j][1]
                    tmp_var=nodeToFeatureToValue_2[i][j][2]
                    
                    nodeToFeatureToValue_1[i].update({feature_val: [ tmp_cnt, tmp_mean, tmp_var] })
 
    return  nodeToFeatureToValue_1


In [10]:

a=(1,2,3,4)
b=(2,4,5,7)

tmparr=[]
tmparr.append(a)
tmparr.append(b)
# print tmparr[:][2]
# print type(a)

col=[row[1] for row in tmparr]
print col

[2, 4]


In [23]:
# split decision

#implementation notes:
# bit-operation to record the set of selected feature values in the left child


def split_onOneFeature( statis_values,sumY_node, count_node, lrcount ,lrest  ):
    
    value_cnt= len(statis_values)
    
    leftY_sum=0
    left_count=0
    
    rightY_sum= sumY_node
    right_count= count_node
    
    bestSplitMetric=  (sumY_node /count_node)*(sumY_node)
    #this is a bit-record variable
    leftSplit_valueSet =0 
    
    
    #..........test
    
#     tmpsum=0.0
#     for i in range(0,value_cnt):
#         tmpsum=tmpsum+  1.0*statis_values[i][1][0]*statis_values[i][1][1]
    
#     if tmpsum == sumY_node:
#         print 'correct'
#     else:
#         print 'problem', tmpsum- sumY_node
    
    
    #..................
    
    
    for i in range(0,value_cnt):
        
        leftY_sum = leftY_sum + statis_values[i][1][1] * statis_values[i][1][0]
        rightY_sum= sumY_node- leftY_sum
        
        left_count=left_count+ statis_values[i][1][0]
        right_count=count_node-left_count
        
        current_feature_value= statis_values[i][0]
        
        leftMetric=0
        rightMetric=0
        if left_count !=0 :
            leftMetric= leftY_sum*leftY_sum/left_count
        if right_count !=0 :
            rightMetric= rightY_sum*rightY_sum/right_count
        
        current_splitMetric= leftMetric + rightMetric
        
        if current_splitMetric > bestSplitMetric:
            
            
            # for debug
            lrcount.append(left_count)
            lrcount.append(right_count)
#             lrvar[0]= left
            
            lrest.append(leftY_sum/left_count)
            lrest.append(rightY_sum/right_count)        
         
            bestSplitMetric=current_splitMetric   
            #bit variable to record which value is chosen for the left child
            leftSplit_valueSet=leftSplit_valueSet+ (1<<current_feature_value)
            
#     if value_cnt==4:
#             print leftSplit_valueSet        
    
#     print (sumY_node /count_node)*(sumY_node), bestSplitMetric


   
    
    return (bestSplitMetric, leftSplit_valueSet)

# def split_onOneNode( statis_featureSplits  ):
    
#     vaue_cnt= len(statis_values)
#     tmp_var_sum=0
#     for i in range(0,vaue_cnt):
#         tmp_var_sum = tmp_var_sum + statis_values[i][1][2]
                
#     return (value, values_leftChild)

def find_bestSplit(local_aggre_nodes, current_NumNodes, nodes_tree, nodes_tree_test):
    
#     currentNode_split_fromMaster.unpersist(blocking = True)
    
    #debug
    count_layer=[]
    
    nodes_layer=[]
    current_idx_nodes =0
    
    lr_count=[]
    
    
    if local_aggre_nodes[ current_idx_nodes ][0] == -1:
        current_idx_nodes=1
    
    
    for i in range(0, current_NumNodes):
        
        
        if local_aggre_nodes[ current_idx_nodes ][0] != i:
            nodes_layer.append( (-1,-1) )
            nodes_tree.append( (-1,-1) )
            
            
            continue
            
        
        tmp_splitValue_oneFeature=[]
        tmp_sumY_node = local_aggre_nodes[i][1]['sumY'] 
        tmp_count_node = local_aggre_nodes[i][1]['count'] 
    
    
        best_splitMetric_feature=(tmp_sumY_node /tmp_count_node)*\
        (tmp_sumY_node )
        best_split_feature=-1
        best_split_featureValueSet=-1
        
        best_leftEst=0
        best_rightEst=0
        
    
        for j in range(0,numFeatures):
            
            satistToValues = local_aggre_nodes[i][1][j] 
        
#         print len(satistTovalues)
        
            sorted_valuesOnMean = sorted(satistToValues.items(), key= \
                                         lambda val: val[1][1] ) 
        
#         if i==0 and j==0:
#             print i,j
#             print (sorted_valuesOnMean)

            #debug
            lr_count=[]
            lr_est=[]

             
            split=split_onOneFeature( sorted_valuesOnMean, tmp_sumY_node,\
                                     tmp_count_node,  lr_count,lr_est )
    
            if split[0] > best_splitMetric_feature:
            
                best_splitMetric_feature=split[0]
                
                best_split_feature = j
                best_split_featureValueSet= split[1]
                
                best_leftEst=lr_est[0]
                best_rightEst=lr_est[1]
                
        
#         print 'node split:',(tmp_sumY_node /tmp_count_node)*(tmp_sumY_node ), best_splitMetric_feature
    
        # for one-layer point assignment
        nodes_layer.append((best_split_feature,best_split_featureValueSet,best_leftEst,best_rightEst))
        
        # for whole-tree point assignment
        nodes_tree.append((best_split_feature,best_split_featureValueSet,best_leftEst,best_rightEst))
        
        
        nodes_tree_test.append(best_leftEst)
        nodes_tree_test.append(best_rightEst)
        
        
        
        current_idx_nodes= current_idx_nodes+1
    
#     print node_split
        
    return nodes_layer
        


In [24]:
# training error inspection 
def error_partition_combiner(list_dvAndfeatures):
    # each feature-value has a triple [cnt, mean, variance]
    
    nodes_dict={}
    tmpcnt=0
    
    for dvAndfeatures in list_dvAndfeatures:
        
        Y= dvAndfeatures[0]
        
        node= search_nodeToData(dvAndfeatures[1:numFeatures+1], \
                                currentNode_split_fromMaster.value)
        
        if node in nodes_dict.keys():
            
            # new added: sum of y in a node
            nodes_dict[node]['sumY'] = nodes_dict[node]['sumY']+Y 
            nodes_dict[node]['count'] = nodes_dict[node]['count']+1 
            
            nodes_dict[node]['sumYsquare'] = nodes_dict[node]['sumYsquare']+ Y*Y
            
        else:
            nodes_dict.update( {node: {}})  
            
            # new added: sum of y in a node
            nodes_dict[node].update( { 'sumY': Y} )
            nodes_dict[node].update( { 'count': 1} )
            nodes_dict[node].update( {'sumYsquare': Y*Y} )
            
                
    return zip(nodes_dict.keys(), nodes_dict.values())


def error_merge_parttion_combiner(nodeToFeatureToValue_1,nodeToFeatureToValue_2 ):
    
    # new added: sum of Y in a node
    nodeToFeatureToValue_1['sumY']= nodeToFeatureToValue_1['sumY']+ \
    nodeToFeatureToValue_2['sumY']
    
    nodeToFeatureToValue_1['count']= nodeToFeatureToValue_1['count']+\
    nodeToFeatureToValue_2['count']
    
    nodeToFeatureToValue_1['sumYsquare']= nodeToFeatureToValue_1['sumYsquare']+ \
    nodeToFeatureToValue_2['sumYsquare']
 
    return  nodeToFeatureToValue_1

In [27]:
# training  process

# current_layer=1

dta_test = dta
nodeSplits_tree=[]

nodes_tree_predict=[]

#ini parameter
currentNode_split_fromMaster = sc.broadcast([])


start = time.time()

for i in range(0,maxdepth):
    
    current_NumNodes= int( math.pow(2,  i))
    
#     print 'current split decision:',currentNode_split_fromMaster.value
    
    
      #----------------------- debug ---------------
    
    error_partitions = dta_test.mapPartitions( error_partition_combiner )

    error_nodes =error_partitions.reduceByKey(lambda statis_partition_1, \
                                              statis_partition_2: 
                              error_merge_parttion_combiner(statis_partition_1,\
                                                      statis_partition_2 ))
    
    local_error_nodes= error_nodes.collect()
    
    local_sorted_error_nodes = sorted(local_error_nodes,key= lambda val:val[0])
    
#     print 'current number of nodes:', len(local_sorted_error_nodes)
    tmpres=[]
    tmpcnt=[]
    for k in range(0, len(local_sorted_error_nodes)):   
            
        tmpvar=local_sorted_error_nodes[k][1]['sumYsquare']/local_sorted_error_nodes[k][1]['count']\
        -(local_sorted_error_nodes[k][1]['sumY']/local_sorted_error_nodes[k][1]['count'])*\
        (local_sorted_error_nodes[k][1]['sumY']/local_sorted_error_nodes[k][1]['count'])
        
        tmpres.append( tmpvar/ 100000.0 )
        tmpcnt.append(local_sorted_error_nodes[k][1]['count'])
        
    print 'before split:',tmpres
    print 'before split:', tmpcnt
    
    #---------------------------------------------
    
    
    statis_partitions = dta_test.mapPartitions( partition_combiner )
    
    aggre_nodes =statis_partitions.reduceByKey(lambda statis_partition_1, statis_partition_2: 
                              merge_parttion_combiner(statis_partition_1,statis_partition_2 ))

    local_aggre_nodes= aggre_nodes.collect()
    
    nodeSplits_layer=find_bestSplit(sorted(local_aggre_nodes,key= lambda val:val[0])\
                                    ,current_NumNodes,nodeSplits_tree, nodes_tree_predict)
    

    
    #for whole tree
    currentNode_split_fromMaster = sc.broadcast(nodeSplits_tree)
    
    
    #----------------------- debug ---------------
    
    error_partitions = dta_test.mapPartitions( error_partition_combiner )

    error_nodes =error_partitions.reduceByKey(lambda statis_partition_1, \
                                              statis_partition_2: 
                              error_merge_parttion_combiner(statis_partition_1,\
                                                      statis_partition_2 ))
    
    local_error_nodes= error_nodes.collect()
   
    
    local_sorted_error_nodes = sorted(local_error_nodes,key= lambda val:val[0])
    
#     print 'current number of nodes:', len(local_sorted_error_nodes)
    tmpres=[]
    tmpcnt=[]
    
    for k in range(0, len(local_sorted_error_nodes)):
        
        tmpweight=0.0
        
        if len(local_sorted_error_nodes) > 1:
            if (k%2) == 0 :
                tmpweight= local_sorted_error_nodes[k][1]['count']+ \
                local_sorted_error_nodes[k+1][1]['count']
            else:
                tmpweight= local_sorted_error_nodes[k][1]['count']+ \
                local_sorted_error_nodes[k-1][1]['count']
            weight=1.0*local_sorted_error_nodes[k][1]['count']/tmpweight
            
        else:
            weight=1.0    
            
#       weight=1.0     
            
        tmpvar=local_sorted_error_nodes[k][1]['sumYsquare']/local_sorted_error_nodes[k][1]['count']\
        -(local_sorted_error_nodes[k][1]['sumY']/local_sorted_error_nodes[k][1]['count'])*\
        (local_sorted_error_nodes[k][1]['sumY']/local_sorted_error_nodes[k][1]['count'])
        
        tmpvar= weight*tmpvar/ 100000.0
        
        tmpres.append( tmpvar )
        tmpcnt.append( local_sorted_error_nodes[k][1]['count'] )
    
    reduced_error=[]
    for k in range(0, len(local_sorted_error_nodes),2):
        reduced_error.append( tmpres[k]+tmpres[k+1]    )
        
    print 'after split:',reduced_error
    print 'after split:',tmpcnt
    
    #---------------------------------------------
    
    
    
    
    
    #for layer of nodes
#     currentNode_split_fromMaster = sc.broadcast(nodeSplits_layer)


end = time.time()
elapsed = end - start

print elapsed,'sec'
    
    

before split: [0.014511236311689229]
before split: [7009728]
after split: [0.014349689349411918]
after split: [3437083, 3572645]
before split: [0.010798210451503318, 0.01776640938312934]
before split: [3437083, 3572645]
after split: [0.010704523535973601, 0.01756699078279276]
after split: [2228171, 1208912, 2495854, 1076791]
before split: [0.008470207290702155, 0.014822638553892788, 0.01428674455227415, 0.025170152097685614]
before split: [2228171, 1208912, 2495854, 1076791]
after split: [0.00841233901993484, 0.014717587106757895, 0.014219434668983032, 0.024912707152249143]
after split: [1057594, 1170577, 946882, 262030, 1427773, 1068081, 831897, 244894]
before split: [0.006656452258178919, 0.00999874922952591, 0.012196111741636973, 0.023829290486815244, 0.012844802687455784, 0.016056994206285077, 0.02144827882450455, 0.03668125816846736]
before split: [1057594, 1170577, 946882, 262030, 1427773, 1068081, 831897, 244894]
after split: [0.006601765099321311, 0.009922251568324374, 0.012101

In [35]:
len(nodes_tree_predict)

62

In [38]:
len(nodeSplits_tree)

31

In [10]:
leaf_nodes_left=int( len(nodes_tree_predict)- math.pow(2,maxdepth))
leaf_nodes_right=len(nodes_tree_predict)
    
leaf_nodes= nodes_tree_predict[leaf_nodes_left:leaf_nodes_right]
len(leaf_nodes)

NameError: name 'nodes_tree_predict' is not defined

In [9]:
a=[1,2,3,4,5,5]
a[1:2]


[2]

In [ ]:
# testing phase

dta_test=dta
dta_test_num=len(dta_test)

err_square=0

for i in range(0, dta_test_num):
    node= search_nodeToData( dta_test[i][1:numFeatures+1], nodeSplits_tree)
    err_square=err_square+(dta_test[i][0]-nodes_tree_predict[node])*(dta_test[i][0]-nodes_tree_predict[node])
    
print err_square/dta_test_num




labeledData=  dta_resetIdx.map(lambda line: LabeledPoint( 
        line[0],[line[1],line[2],line[3],
        line[4], line[5],line[6],line[7]]))


(datasubset, test_subset) = labeledData.randomSplit([0.005, 0.2])


In [12]:
tmpsum=0
featurecnt=0

for j in local_aggre_nodes[0][1][2].keys():
    featurecnt= featurecnt+local_aggre_nodes[0][1][2][j][0]
#     for i in local_aggre_nodes[0][1][0][j].keys():
#         tmpsum=tmpsum + local_aggre_nodes[0][1][0][j][i]
    
print featurecnt,tmpsum

3315067 0


In [706]:
# def search_nodeToData(features, tree):
        
#     nodeNum=len(tree)
    
#     if nodeNum == 0:
#         return 0;
    
#     current_nodeIdx=0
#     while current_nodeIdx< nodeNum:
#         split_feature= tree[current_nodeIdx][0]
#         split_valueSet= tree[current_nodeIdx][1]
        
#         if split_feature==-1:
#             return -1
        
#         if (( 1<<features[split_feature]) &  split_valueSet) != 0 :
#             current_nodeIdx=current_nodeIdx*2+1
#         else:
#             current_nodeIdx=current_nodeIdx*2+2
    
# #     return random.randint(0, 2)    
#     return current_nodeIdx - nodeNum


#test: one-pass method
dta_test = dta

#tree=currentNode_split_fromMaster.value()

start = time.time()

statis_partitions = dta_test.mapPartitions( partition_combiner )
# local_statis_partition=statis_partitions.collect()
aggre_nodes =statis_partitions.reduceByKey(lambda statis_partition_1, statis_partition_2: 
                              merge_parttion_combiner(statis_partition_1,statis_partition_2 ))
local_aggre_nodes= aggre_nodes.collect()


end = time.time()
elapsed = end - start

print elapsed,'sec'

(0, {'count': 3315067, 'sumY': 2758054210.0, 'sumYsquare': 161259522932250.0})

In [320]:
# for debug

                    #two-pass           one-pass
1 node(root):      36.1043100357 sec , 35.2387490273 sec

2 nodes:           36.1584570408 sec , 35.16437006 sec

4 nodes:           36.6243078709 sec ,  35.6500589848 sec

8 nodes:           36.7351138592 sec,   35.6860120296 sec

16 nodes:          36.8523671627 sec,   35.8512570858 sec
 
32 nodes:          37.0641298294 sec,   37.1946568489 sec
    
64 nodes:          37.4132461548 sec,   38.327684164 sec


7


In [411]:
tmpdict={ 1:[2,'a'], 2:[1,'b'], 4:[8,'c']     }

print tmpdict.items()

sorted(tmpdict.items(), key= lambda val: val[1][0] )




[(1, [2, 'a']), (2, [1, 'b']), (4, [8, 'c'])]


[(2, [1, 'b']), (1, [2, 'a']), (4, [8, 'c'])]

In [18]:
# for debug 
tmparr=()
tmparr={ 'faf':2,1:2,4:2}
del tmparr['faf']
tmparr

tmpddict={ 1:10,2:30}

def tmpfunc(tmpdict):
    
    ldict=tmpdict.copy()
    ldict[1]=ldict[1]-10
    return ldict

tmpfunc(tmpddict)
tmpfunc(tmpddict)

print tmpddict

{1: 10, 2: 30}
